# Consultas SQL basicas SELECT

En esta sección, aprenderemos como extraer datos de nuestra base de datos.

Primero, vamos a cargar una base de datos que descargaremos de la plataforma Kaggle. Kaggle es una plataforma en línea que provee recursos relacionados con ciencia de datos, aprendizaje automático y análisis de datos. En Kaggle, podremos encontrar muchos *datasets* (conjunto de datos) educacionales. Nosotros descargaremos un *dataset* y lo cargaremos en una base de datos en SQLite 3 para hacer consultas sobre ella. 

Usaremos la siguiente base de datos, que recopila las canciones más populares del mundo en Spotify: https://www.kaggle.com/datasets/arnavvvvv/spotify-music.

Necesitarás crear una cuenta para descargar el archivo (el archivo será de tipo CSV).

Exploremos el archivo CSV con Pandas:

In [ ]:
import pandas as pd

spotify_df = pd.read_csv('Popular_Spotify_Songs.csv', encoding='latin1')
spotify_df.head()

Tenemos 953 canciones en total:

In [ ]:
spotify_df.shape

In [ ]:
spotify_df.columns

# Conexión a la base de datos

Vamos a cargar los datos del archivo CSV a un base de datos. Para poder realizar consultas sobre una base de datos, antes debemos crear una conexión a la base de datos. Vamos a crear una base de datos con nombre "spotifydb" y conectarnos a ella:

In [ ]:
# La librería sqlite3 nos permitirá conectarnos a bases de datos SQLite

import sqlite3

# Creamos una conexión a 'spotifydb'. Si no existe la base de datos, crea la base de datos
conn = sqlite3.connect('spotifydb')

# Nos ayudará a ejecutar acciones, operaciones y consultas sobre la base de datos
cursor = conn.cursor()

Nos podemos fijar que se ha creado un archivo con nombre 'spotifydb' en la carpeta. Ahi se guardarán todos los datos de la base de datos.

# Creación de una tabla en la base de datos

Las bases de datos relacionales están organizadas en forma de tablas. Cada tabla está compuesta por filas y columnas.

1. Las columnas (también llamadas campos) representa un atributo específico de la entidad que la tabla describe. Por ejemplo, en una tabla de canciones de Spotify, las columnas pueden ser el nombre de la canción, el artista, año de lanzamiento, etc.
2. Las filas (también llamadas registros) representa una instancia única de la entidad que una tabla describe. Por ejemplo, una tabla de canciones de Spotify, cada fila representará a una canción distinta con cada columna que contiene información de esa canción.

Vamos a crear una tabla para cargar los datos del archivo CSV. Solo vamos a tener en cuenta algunas columnas (o campos) para esta sección. 

Sintaxis para la creación de una tabla (si no existe la tabla en la base de datos):

```SQL

CREATE TABLE table_name(
    column1_name column1_type,
    column2_name column2_type,
    column3_name column3_type
)

```

Al igual que en Python, en SQLite existen distintos tipos de datos para las columnas. Algunas pueden contener número enteros (INTEGER), números reales (REAL), texto (TEXT), etc. Aquí tienes una referencia más detallada sobre los tipos de datos en SQLite: https://www.sqlite.org/datatype3.html.

Primero vamos a filtrar solamente las siguientes columnas:

- Track name (Nombre de la canción)
- Artist(s) name (Nombre del artista)
- Released year (Año de lanzamiento)
- Released month (Mes de lanzamiento)
- Released day (Día de lanzamiento)
- In Spotify Playlists (Cantidad de playlists donde se encuentra la canción)
- Streams (Cantidad de reproducciones de la canción en Spotify)

In [ ]:
# Primero filtramos las columnas que usaremos 
spotify_df = pd.read_csv('Popular_Spotify_Songs.csv')

columns = ['track_name', 'artist(s)_name', 'released_year', 'released_month', 'released_day', 'in_spotify_playlists', 'streams']
spotify_df = spotify_df[columns]

In [ ]:
spotify_df.head()

Ahora vamos a crear una nueva columna llamada `released_date` donde juntaremos el año, mes y día de lanzamiento en un único campo:

In [ ]:
spotify_df['released_date'] = spotify_df['released_year'].astype('str') + '-' + spotify_df['released_month'].astype('str') + '-' + spotify_df['released_day'].astype('str')
spotify_df.drop(columns=['released_year', 'released_month', 'released_day'], inplace=True)
spotify_df

Además, cambiaremos el nombre de la columna `artist(s)_name` por `artist_name` ya que SQLite no acepta caracteres espaciales (como paréntesis, corchetes) en el nombre de sus columnas.

In [ ]:
spotify_df.rename(columns={'artist(s)_name' : 'artist_name'}, inplace=True)
spotify_df

Nos quedamos con 5 columnas en nuestro dataframe que insertaremos en la base de datos.

Creamos una tabla en la base de datos con estos 5 campos con sus respectivos tipos de datos:

In [ ]:
# Ejecutamos la query con el método 'execute' de cursor. La query es una cadena.
# Si la tabla ya existe en la base de datos, nos generará un error

cursor.execute('''CREATE TABLE spotify_songs_table (
                    track_name TEXT,
                    artist_name TEXT,
                    in_spotify_playlists INTEGER,
                    streams INTEGER,
                    released_date DATE
              )''')


Ya tenemos creada una tabla en nuestra base de datos, pero esta tabla se encuentra vacía. Podemos insertar registros a una tabla de manera sencilla con pandas, pero el nombre de las columnas de la tabla con las columnas del DataFrame deben ser **iguales**:

In [ ]:
# Parámetros:
# - Nombre de la tabla: spotify_songs_table
# - Conexióna a la base da datos: conn
# - if_exists = 'append': Si la tabla ya existe, insertamos los datos en esa tabla
# - index = False : El índice del DataFrame no se inserta a la base de datos

spotify_df.to_sql('spotify_songs_table', conn, if_exists='append', index=False)

Con esto, nuestra tabla tiene 953 registros.

Cerramos la conexión a la base de datos:

In [ ]:
conn.close()

# Instrucción SELECT

Nuestra base de datos ya se encuentra cargada con los registros del archivo CSV que descargamos de la plataforma Kaggle. 

Ahora veamos como extraer registros de la base de datos. Primero nos conectamos a la base de datos:

In [ ]:
conn = sqlite3.connect('spotifydb')
cursor = conn.cursor()

La intrucción `SELECT` nos ayudará a extraer **todos** los registros de una tabla. La sintaxis de la instrucción es:

```SQL

SELECT *
FROM table_name

```

In [ ]:
cursor.execute('''SELECT *
                  FROM spotify_songs_table''')

Ya ejecutamos la instrucción `SELECT`, ahora recuperaremos los registros con el método `fetchall()`:

In [ ]:
rows = cursor.fetchall()
rows

Obtenemos una lista de tuplas, donde cada tupla representa un registro. 

## Sentencia WHILE

Ahora imaginemos que solo queremos obtener los registros de un artista en concreto. Para ello usaremos la sentencia `WHILE` que nos permite poner condicionales dentro de una instrucción `SELECT`:

```SQL

SELECT *
FROM table_name
WHERE condition

```

Extraigamos todos los registros de el artiste *Drake*:

In [ ]:
# Selecciona todos los registros donde su campo 'artist_name' sea igual a 'Drake'
# Drake va entre comillas, pues al igual que en Python, para indicar
# que un objeto se trata de una cadena, va entre comillas

cursor.execute('''SELECT * 
                  FROM spotify_songs_table
                  WHERE artist_name = 'Drake'
            ''')

rows = cursor.fetchall()
rows

Hay algunas canciones dentro de la tabla que tienen múltiples artistas. Por ejemplo, la siguiente canción:

In [ ]:
cursor.execute('''SELECT * 
                  FROM spotify_songs_table
                  WHERE track_name = 'Privileged Rappers'
            ''')

rows = cursor.fetchall()
rows

Si queremos obtener todas las canciones donde *Drake* tuvo participación, en vez de que la condición sea `artist_name = 'Drake'`, pondremos `artist_name LIKE '%Drake%'`. 

In [ ]:
cursor.execute('''SELECT * 
                  FROM spotify_songs_table
                  WHERE artist_name LIKE '%Drake%'
            ''')

rows = cursor.fetchall()
rows

La condición `columna LIKE '%cadena%'` es verdadera si `cadena` está contenido en `columna` del registro. A diferencia de `columna = cadena`, donde `cadena` tiene que ser igual a `columna`. Por ello, la condición `artist_name LIKE '%Drake%'` toma todos los registros donde **Drake** está contenido en la columan `artist_name`.

También podemos hacer condicionales sobre comparaciones de números:

In [ ]:
# Todas las canciones con al menos 500000000 reproducciones en spotify

cursor.execute('''SELECT * 
                  FROM spotify_songs_table
                  WHERE streams >= 500000000
            ''')

rows = cursor.fetchall()
rows

También podemos comparar fechas:

In [ ]:
# Todas las canciones que tuvieron una fecha de lanzamiento mayor o igual al 1 de Enero de 2023

cursor.execute('''SELECT * 
                  FROM spotify_songs_table
                  WHERE released_date >= '2023-01-01'
            ''')

rows = cursor.fetchall()
rows

Podemos combinar varias condicionales con AND y OR:

In [ ]:
# Todas las canciones con al menos 500000000 reproducciones en spotify y donde Drake tuvo participación

cursor.execute('''SELECT * 
                  FROM spotify_songs_table
                  WHERE artist_name LIKE '%Drake%' AND
                        streams >= 500000000
            ''')

rows = cursor.fetchall()
rows

In [ ]:
# Todas las canciones donde el artista es 'Drake' o 'Miley Cyrus'

cursor.execute('''SELECT * 
                  FROM spotify_songs_table
                  WHERE artist_name = 'Drake' OR
                        artist_name = 'Miley Cyrus'
            ''')

rows = cursor.fetchall()
rows

## Seleccionando columnas

Hasta ahora, hemos extraído los valores de todas las columnas de cada registro. Pero tenemos la opción de escoger solo algunas columnas. En vez de poner `*` al lado de `SELECT`, especificamos el nombre de las columnas que queremos extraer.

``` SQL

SELECT column1, column2, column3
FROM table_name
WHERE condition

```

In [ ]:
# Solo extraeremos el nombre de la canción y la fecha de lanzamiento de cada registro

result = cursor.execute('''SELECT track_name, released_date
                              FROM spotify_songs_table
                              WHERE artist_name LIKE '%Drake%'
                        ''')

rows = cursor.fetchall()
rows

La librería Pandas ofrece una función (`read_sql_query`) que permite ejecutar una `query` de SQLite y guardar los registros obtenidos en un DataFrame.

In [ ]:
query = '''SELECT *
            FROM spotify_songs_table
            WHERE artist_name LIKE '%Drake%'
        '''

result_df = pd.read_sql_query(query, conn)
result_df

In [ ]:
query = '''SELECT track_name, released_date
            FROM spotify_songs_table
            WHERE artist_name LIKE '%Drake%'
        '''

result_df = pd.read_sql_query(query, conn)
result_df

Dentro de la instrucción `SELECT` podemos cambiar el nombre de las columnas con la sentencia `AS` (esto no cambia el nombre de las columnas en la tabla, solo el resultado presentará otros nombres de columnas).

In [ ]:
# Cambiamos el nombre de la columna 'track_name' por 'Nombre de la canción' y 'released_date' por 'Fecha de lanzamiento'

query = '''SELECT track_name AS 'Nombre de la canción',
                  artist_name AS 'Artista',
                  released_date AS 'Fecha de lanzamiento'
            FROM spotify_songs_table
            WHERE artist_name LIKE '%Drake%'
        '''

result_df = pd.read_sql_query(query, conn)
result_df

## Ordenando los registros con ORDER BY

Los registros obtenidos de una `query` en SQLite podemos ordenarlas con `ORDER BY`:

In [ ]:
# Ordenando por 'streams'

query = '''SELECT *
            FROM spotify_songs_table
            WHERE artist_name LIKE '%Drake%'
            ORDER BY streams
            '''

result_df = pd.read_sql_query(query, conn)
result_df

También podemos ordenar de forma descendente si ponemos la sentencia `DESC` a la derecha de la nombre la columna:

In [ ]:
# Ordenando por 'released_date' de forma descendente

query = '''SELECT *
            FROM spotify_songs_table
            WHERE artist_name LIKE '%Drake%'
            ORDER BY released_date DESC
            '''

result_df = pd.read_sql_query(query, conn)
result_df

Podemos ordenar en base de dos columnas (se ordenará de izquierda a derecha):

In [ ]:
# Primero ordena por 'released_date' y luego por 'streams'
# Esto quiere decir que primero ordenará todos los registros por 'released_date', y si dos registros tienen igual 'released_date' entonces se ordenará por 'streams'

query = '''SELECT *
            FROM spotify_songs_table
            WHERE artist_name LIKE '%Drake%'
            ORDER BY released_date, streams
            '''

result_df = pd.read_sql_query(query, conn)
result_df